<h1 style='font-size:40px'> PySpark Tutorial</h1>
<h2 style='font-size:30px'>Instalação </h2>

In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


<div> 
    <hr>
    <h2 style='font-size:30px'> Introdução</h2>
</div>

In [3]:
# Por efeitos de comparação, vamos carregar um arquivo csv no pandas
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/krishnaik06/Pyspark-With-Python/main/test1.csv')
df.to_csv('test1.csv', columns=['Name', 'age', 'Experience', 'Salary'], index=False)

<div> 
    <ul style='font-size:20px'>
        <li>
            Iremos agora criar uma sessão no Spark. Para isso, é necessário importar o objeto SparkSession, de pyspark.sql.
        </li>
    </ul>
</div>
<strong style='font-size:16px'>Nota: para se usar o pyspark, é necessário ter o Java instalado em sua máquina! </strong>

In [4]:
# A seção deve ter um determinado nome.
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate() 
spark

22/04/30 19:18:10 WARN Utils: Your hostname, veiga-Inspiron resolves to a loopback address: 127.0.1.1; using 192.168.15.21 instead (on interface wlp7s0)
22/04/30 19:18:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 19:18:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Lendo um arquivo csv. Note como a sintaxe é semelhante com a do pandas.
# O pyspark, por padrão, não considera a primeira linha do csv como o header. Para que isso aconteça, defina o argumento
# 'header', de 'read.csv' como 'true'.

# Veja, os argumentos booleanos aceitam também strings.
df_pyspark = spark.read.csv('test1.csv', header='true')

# Outra maneira de se fazer uma leitura customizada dos dados.
df_pyspark_option = spark.read.option('header', 'true').csv('test1.csv')

# Logo de cara, já notamos a estética distinta dos DataFrames do pyspark.
df_pyspark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
# A biblioteca tem inúmeras semelhanças com o pandas.
df_pyspark.head(3) ,df_pyspark.columns

([Row(Name='Krish', age='31', Experience='10', Salary='30000'),
  Row(Name='Sudhanshu', age='30', Experience='8', Salary='25000'),
  Row(Name='Sunny', age='29', Experience='4', Salary='20000')],
 ['Name', 'age', 'Experience', 'Salary'])

<div> 
    <hr>
    <h2 style='font-size:30px'> DataFrames</h2>
</div>
<h3 style='font-size:30px;font-style:italic'>Schema </h3>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Por padrão, todas colunas dos DataFrames pyspark são tidas como strings.
        </li>
    </ul>
</div>

In [7]:

# Criando outra seção no pyspark
spark = SparkSession.builder.appName('Practice').getOrCreate()

# Lendo novamete o test1.csv
df_pyspark = spark.read.csv('test1.csv', header='true')

# Veja, mesmo as colunas que queremos que sejam numéricas estâo como texto.
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [8]:
# Neste caso read.csv tem o argumento inferSchema, capaz de inferir o tipo de dado das colunas.
df_pyspark = spark.read.csv('test1.csv', header='true', inferSchema='true')
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



<h3 style='font-size:30px;font-style:italic'> Select</h3>
<div> 
    <ul style='font-size:20px'> 
        <li>
            A seleção de colunas específicas dos DataFrames é um pouco diferente no pyspark. Para essa operação, é necessário usar o método <em>select </em>.
        </li>
    </ul>
</div>

In [9]:
df_pyspark = spark.read.csv('test1.csv', header=True, inferSchema=True)

# Coletando apenas o nome do funcionário e seu salário.
df_pyspark.select(['Name', 'Salary']).show()

+---------+------+
|     Name|Salary|
+---------+------+
|    Krish| 30000|
|Sudhanshu| 25000|
|    Sunny| 20000|
|     Paul| 20000|
|   Harsha| 15000|
|  Shubham| 18000|
+---------+------+



In [10]:
# Outro método do pyspark muito similar ao do pandas é 'describe'.
df_pyspark.describe().show()

+-------+------+------------------+-----------------+------------------+
|summary|  Name|               age|       Experience|            Salary|
+-------+------+------------------+-----------------+------------------+
|  count|     6|                 6|                6|                 6|
|   mean|  null|26.333333333333332|4.666666666666667|21333.333333333332|
| stddev|  null| 4.179314138308661|3.559026084010437| 5354.126134736337|
|    min|Harsha|                21|                1|             15000|
|    max| Sunny|                31|               10|             30000|
+-------+------+------------------+-----------------+------------------+



<h3 style='font-size:30px;font-style:italic'> Adicionando e Removendo Colunas</h3>

In [11]:
# A sintaxe de adição de novas colunas é diferente da do pandas.

# 'withColumn' não modifica o DF inplace.
df_pyspark.withColumn('Ages with no Experience', df_pyspark['age'] - df_pyspark['Experience']).show()

+---------+---+----------+------+-----------------------+
|     Name|age|Experience|Salary|Ages with no Experience|
+---------+---+----------+------+-----------------------+
|    Krish| 31|        10| 30000|                     21|
|Sudhanshu| 30|         8| 25000|                     22|
|    Sunny| 29|         4| 20000|                     25|
|     Paul| 24|         3| 20000|                     21|
|   Harsha| 21|         1| 15000|                     20|
|  Shubham| 23|         2| 18000|                     21|
+---------+---+----------+------+-----------------------+



In [12]:
# Por outro lado, drop é idêntico ao pandas. Novamente, o método não modifica o DF inplace.
df_pyspark.drop('Experience').show()

+---------+---+------+
|     Name|age|Salary|
+---------+---+------+
|    Krish| 31| 30000|
|Sudhanshu| 30| 25000|
|    Sunny| 29| 20000|
|     Paul| 24| 20000|
|   Harsha| 21| 15000|
|  Shubham| 23| 18000|
+---------+---+------+



In [13]:
# Renomeando colunas (também não é inplace).
df_pyspark.withColumnRenamed('Name', 'New Name').show()

+---------+---+----------+------+
| New Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



<h2 style='font-size:30px'> Lidando com valores perdidos</h2>

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()
spark

In [15]:
# Neste novo DataFrame, é possível enxergar valores nulos.
df_pyspark = spark.read.csv('test2.csv', header=True, inferSchema=True).drop('_c0')
df_pyspark.show()

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubham|23.0|       2.0|18000.0|
|   Mahesh|null|      null|40000.0|
|     null|34.0|      10.0|38000.0|
|     null|36.0|      null|   null|
+---------+----+----------+-------+



<h3 style='font-size:30px;font-style:italic'>Drop</h3>

In [16]:
# Droppando valores nulos.
# O atributo 'na' dos DataFrames do pyspark possui uma série de métodos para lidar com Nan.

# Lembrando, os métodos não são inplace.

# 'how' pode ser 'any' ou 'all'.
# thres define o número mínimo de NaN's que a linha tem que ter para ser excluída.
df_pyspark.na.drop(how='any', thresh=2).show()

# 'subset' diz ao pyspark considerar apenas os nulos da coluna passada como argumento.
df_pyspark.na.drop(how='any', subset='Experience').show()

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubham|23.0|       2.0|18000.0|
|   Mahesh|null|      null|40000.0|
|     null|34.0|      10.0|38000.0|
+---------+----+----------+-------+

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubham|23.0|       2.0|18000.0|
|     null|34.0|      10.0|38000.0|
+---------+----+----------+-------+



<h3 style='font-size:30px;font-style:italic'>Fill</h3>

In [19]:
# Substituindo os nulos por outra coisa (fill).
# Podemos também escolher um subset do DataFrame para fazer a operação.
df_pyspark.na.fill('Missing', subset=['Experience']).show()

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubham|23.0|       2.0|18000.0|
|   Mahesh|null|      null|40000.0|
|     null|34.0|      10.0|38000.0|
|     null|36.0|      null|   null|
+---------+----+----------+-------+



<h3 style='font-size:30px;font-style:italic'>Impute</h3>

In [27]:
# Equivalente ao imputer do sklearn.
from pyspark.ml.feature import Imputer
imputer = Imputer(strategy='median', inputCols=['Experience', 'age'],
                 outputCols=[f'{col}_imputed' for col in ['Experience', 'age']])

# Mas, estranhamente, as colunas com valores nulos são mantidas, enquanto são criadas outras com os valores corrigidos.
# O nome das novas colunas é definido pelo argumento outputCols.
imputer.fit(df_pyspark).transform(df_pyspark).show()

+---------+----+----------+-------+------------------+-----------+
|     Name| age|Experience| Salary|Experience_imputed|age_imputed|
+---------+----+----------+-------+------------------+-----------+
|    Krish|31.0|      10.0|30000.0|              10.0|       31.0|
|Sudhanshu|30.0|       8.0|25000.0|               8.0|       30.0|
|    Sunny|29.0|       4.0|20000.0|               4.0|       29.0|
|     Paul|24.0|       3.0|20000.0|               3.0|       24.0|
|   Harsha|21.0|       1.0|15000.0|               1.0|       21.0|
|  Shubham|23.0|       2.0|18000.0|               2.0|       23.0|
|   Mahesh|null|      null|40000.0|               4.0|       29.0|
|     null|34.0|      10.0|38000.0|              10.0|       34.0|
|     null|36.0|      null|   null|               4.0|       36.0|
+---------+----+----------+-------+------------------+-----------+



<h2 style='font-size:30px'> Filter Operation </h2>
<div> 
    <ul style='font-size:20px'>
        <li>
            O comando filter é equivalente à cláusula WHERE do SQL. É com ele que podemos fazer filtragens no DataFrame.
        </li>
    </ul>
</div>

In [32]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Pracice').getOrCreate()
df_pyspark=spark.read.csv('test1.csv', header=True, inferSchema=True)

# Nota: podemos definir o número de linhas a serem printadas com o argumento 'n' de 'show'.
df_pyspark.show(n=5)

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
+---------+---+----------+------+
only showing top 5 rows



In [37]:
# Observe, a estrututa é muito similar à utilizada no SQL.
(df_pyspark.select(['Experience', 'age'])
            .filter((df_pyspark['Salary'] >=25000) & (df_pyspark['age'] >30))
            .show())

+----------+---+
|Experience|age|
+----------+---+
|        10| 31|
+----------+---+



In [47]:
# O pyspark também suporta o uso do ~ como negação.
df_pyspark.filter(~(df_pyspark['age'] > 25)).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



<h2 style='font-size:30px'> Groupby and Aggregation Function </h2>

In [54]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()
df_pyspark=spark.read.csv('test3.csv', header=True, inferSchema=True).drop('_c0')
df_pyspark.show(n=5)

+------+------------+------+
|  Name| Departments|salary|
+------+------------+------+
| Krish|Data Science| 10000|
| Krish|         IOT|  5000|
|Mahesh|    Big Data|  4000|
| Krish|    Big Data|  4000|
|Mahesh|Data Science|  3000|
+------+------------+------+
only showing top 5 rows



<h3 style='font-size:30px;font-style:italic'>GroupBy</h3>

In [55]:
# A sintaxe é praticamente igual à do pandas.
df_pyspark.groupBy('Departments').mean().show()

+------------+-----------+
| Departments|avg(salary)|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



<h3 style='font-size:30px;font-style:italic'>Agg</h3>
<div> 
    <ul style='font-size:20px'>
        <li>
            agg basicamente aplica uma função a uma coluna por inteiro.
        </li>
    </ul>
</div>

In [59]:
# Obtendo a soma dos salários.
df_pyspark.agg({'Salary':'sum'}).show()

+-----------+
|sum(Salary)|
+-----------+
|      73000|
+-----------+



<h2 style='font-size:30px'> ML I</h2>
<div> 
    <ul style='font-size:20px'> 
        <li>
            Vamos entender um pouco do uso do Pyspark em Machine Learning.
        </li>
    </ul>
</div>

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ML').getOrCreate()
spark

22/05/02 19:40:54 WARN Utils: Your hostname, veiga-Inspiron resolves to a loopback address: 127.0.1.1; using 192.168.15.21 instead (on interface wlp7s0)
22/05/02 19:40:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/02 19:40:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Com este DF, criaremos um modelo de Regressão simples de previsão de salários.
training = spark.read.csv('test1.csv', header=True, inferSchema=True)
training.show(n=3)

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
+---------+---+----------+------+
only showing top 3 rows



<div> 
    <ul style='font-size:20px'> 
        <li>
            A biblioteca tem uma dinâmica um pouco distinta no pré-processamento de dados. 
        </li>
    </ul>
</div>

In [9]:
# As variáveis dependentes devem ser agrupadas em uma única coluna com o uso do objeto VectorAssembler.
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent Variables')
output=featureassembler.transform(training)

In [10]:
# O output da transformação feita é uma coluna em que cada linha há uma lista com as variáveis independentes.
output.show(n=3)

+---------+---+----------+------+---------------------+
|     Name|age|Experience|Salary|Independent Variables|
+---------+---+----------+------+---------------------+
|    Krish| 31|        10| 30000|          [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|           [30.0,8.0]|
|    Sunny| 29|         4| 20000|           [29.0,4.0]|
+---------+---+----------+------+---------------------+
only showing top 3 rows



In [11]:
# Ao final, basta criar um novo DataFrame, cujas colunas sejam as com as variáveis independentes e os valores-alvo.
finalized_data = output.select(['Independent Variables', 'Salary'])
finalized_data.show(n=3)

+---------------------+------+
|Independent Variables|Salary|
+---------------------+------+
|          [31.0,10.0]| 30000|
|           [30.0,8.0]| 25000|
|           [29.0,4.0]| 20000|
+---------------------+------+
only showing top 3 rows



In [12]:
# Quebrando o dataset em conjuntos de treino e teste.
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])

In [18]:
from pyspark.ml.regression import LinearRegression

# Ao definir o algoritmo, temos que passar a coluna de features e a de target value como argumentos.
regressor = LinearRegression(featuresCol='Independent Variables', labelCol='Salary')

# Temos que redefinir 'regressor' como 'regressor.fit()'
regressor = regressor.fit(train_data)

22/05/02 19:59:15 WARN Instrumentation: [de309da4] regParam is zero, which might cause numerical instability and overfitting.


In [22]:
# Coeficiente e ponto de interceptação.
print(regressor.coefficients, regressor.intercept)

[-12.495194156093755,1014.994232987314] 17254.9019607843


In [27]:
# Eis aqui o a raiz quadrada do mean squared error das previsões do algoritmo.
from numpy import sqrt
sqrt(regressor.evaluate(test_data).meanSquaredError)

2227.2564118627806

In [25]:
# Estimativas.
regressor.evaluate(test_data).predictions.show()

+---------------------+------+------------------+
|Independent Variables|Salary|        prediction|
+---------------------+------+------------------+
|           [21.0,1.0]| 15000|18007.497116493647|
|           [23.0,2.0]| 18000| 18997.50096116877|
|           [29.0,4.0]| 20000| 20952.51826220684|
|          [31.0,10.0]| 30000|27017.493271818534|
+---------------------+------+------------------+



<h2 style='font-size:30px'> ML II</h2>

<p style='color:red'> Parei em 1:02:40 (começar ML)</p>